# IEEE-CIS Fraud Detection - data analisys

In [1]:
import sys
import warnings
import pandas as pd
import numpy as np
from sklearn import model_selection, preprocessing, ensemble, metrics
#linear_model, naive_bayes, metrics, svm, decomposition
#import xgboost 


warnings.simplefilter('ignore')
__location__ = sys.path[0] 
output_path = f'{__location__}/Input/ieee-fraud-detection/sample_submission.csv'

In [ ]:
#train_identity = pd.read_csv(f'{__location__}/Input/ieee-fraud-detection/train_identity.csv')
train_transaction = pd.read_csv(f'{__location__}/Input/ieee-fraud-detection/train_transaction.csv')
#test_identity = pd.read_csv(f'{__location__}/Input/ieee-fraud-detection/test_identity.csv')
#test_transaction = pd.read_csv(f'{__location__}/Input/ieee-fraud-detection/test_transaction.csv')

In [ ]:
non_num_cols = [train_transaction.columns[n] for n,i in enumerate(train_transaction.dtypes)
                if i not in ('int64','float64')]
#train_transaction[non_num_cols]

In [ ]:
# label encode the target variable 
encoder = preprocessing.LabelEncoder()


for i in non_num_cols:
    #Filling missing string values with most common value
    train_transaction['mod_'+i] = encoder.fit_transform(train_transaction[i].fillna(train_transaction[i].mode()[0]))
    #test_transaction['mod_'+i] = encoder.fit_transform(test_transaction[i].fillna(test_transaction[i].mode()[0])

    

In [ ]:
col_list = [x for x in train_transaction.columns if x not in non_num_cols+['isFraud']]

x_train, y_train, x_valid, y_valid = model_selection.train_test_split(train_transaction[col_list].fillna(0)
                                                                      , train_transaction['isFraud']
                                                                      , test_size=0.9
                                                                     )

In [ ]:
max_features = list(range(1,train.shape[1]))

for max_feature in max_features:
    # RF
    RF = ensemble.RandomForestClassifier(n_estimators=31
                                         , n_jobs=-1
                                         , max_depth=31
                                         , min_samples_split = 0.3
                                         , max_features = max_feature
                                        )
    RF.fit(x_train, x_valid)
    predictions = RF.predict(y_train)
    accuracy = metrics.accuracy_score(predictions, y_valid)
    print (f"RF, min_samples_leafs={min_samples_leaf}: ", accuracy)

    # RF
#    RF = ensemble.RandomForestClassifier(n_estimators=50, min_samples_leaf=25)
#    RF.fit(x_train, x_valid)
#    predictions = RF.predict(y_train)
#    accuracy = metrics.accuracy_score(predictions, y_valid)
#    print ("RF: ", accuracy)
